# P2G for backtranslation data

In [1]:
import sys
print(sys.executable)

/data/ephemeral/.conda/bin/python


## 모델 불러오기

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
import torch

In [3]:
model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector',cache_dir='/data/ephemeral/tmp')
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')

typos_corrector = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    framework="pt",
)

config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

In [4]:
input_text = "완죤 어이업ㅅ네진쨬ㅋㅋㅋ"
output_text = typos_corrector("맞춤법을 고쳐주세요: " + input_text,
            max_length=128,
            num_beams=5,
            early_stopping=True)[0]['generated_text']
print(output_text)

완전 어이없네 진짜 ᄏᄏᄏᄏ.


In [5]:
def p2g(sent):
    output = typos_corrector("맞춤법을 고쳐주세요: " + sent,
            max_length=50,
            num_beams=5,
            early_stopping=True)[0]['generated_text']
    return output

## 데이터 불러오기

In [8]:
!ls

EDA.ipynb			    back_translation_p2g.ipynb	   build
README.md			    back_translation_papago.ipynb  dist
back_translation_googletrans.ipynb  back_translation_pororo.ipynb  py-hanspell
back_translation_hanspell_t5.ipynb  back_translation_t5.ipynb


In [9]:
import os
BASE_DIR = "/data/ephemeral/level2-nlp-datacentric-nlp-06/"
DATA_DIR = os.path.join(BASE_DIR, 'data/')

In [11]:
# 데이터 불러오기
import pandas as pd

train =  pd.read_csv(os.path.join(DATA_DIR, 'hanspell_train_split.csv'))
google = pd.read_csv(os.path.join(DATA_DIR, 'hanspell_backtrans_google.csv'))
t5 = pd.read_csv(os.path.join(DATA_DIR, 'hanspell_backtrans_T5.csv'))

In [12]:
train.shape, google.shape, t5.shape

((4900, 5), (4900, 5), (4900, 5))

In [13]:
train.head()

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트 타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공방어 시스템 공개 안보 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한 유엔 채택 인권결의에 모든 대응조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [15]:
google.head()

,ID,text,target,url,date
0,ynat-v1_train_04767,해외 산악요트 탑승 중 권총 밀수를 위한 출입국 관리소,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,"새로운 유전자 가위 기능 확인, 유전질환 치료 기대",0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 삼단의 일러스트,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란의 새로운 대공 방어 시스템으로 공공 안보를 위한 군사력 강화,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,"북한, 유엔 인권결의안에 반대하는 모든 대응조치 강행",6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [19]:
t5.head()

,ID,text,target,url,date
0,ynat-v1_train_04767,해외에서 구입한 요트에서 권총을 밀수하는 출입국 관리소 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,새로운 유전자 가위의 기능 확인 유전 질환 치료를 기대합니다,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐에 있는 신사의 사진,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란의 새로운 대공방어 체계를 개방하고 확보하기 위해 군사력을 증강한다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,이들이 채택한 인권 결의안에 대한 모든 대응책을 강구하는 등 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


## p2g 적용

In [20]:
# train_df의 text에 apply를 통해 p2g 함수를 적용한 결과를 새로운 컬럼인 p2g_text에 저장
from tqdm import tqdm
tqdm.pandas()
train['text'] = train['text'].progress_apply(p2g)


  0%|          | 10/4900 [00:01<16:51,  4.84it/s]/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 4900/4900 [16:43<00:00,  4.88it/s]


In [22]:
train['text'] = train['text'].apply(lambda x: x.replace('.', ''))

In [23]:
train.head()

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트 타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공 방어 시스템 공개 안 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한유엔 채택 인권결의에 모든 대응 조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [24]:
google['text'] = google['text'].progress_apply(p2g)
google['text'] = google['text'].apply(lambda x: x.replace('.', ''))
google.head()

  0%|          | 0/4900 [00:00<?, ?it/s]/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 4900/4900 [16:23<00:00,  4.98it/s]


,ID,text,target,url,date
0,ynat-v1_train_04767,해외 산악 요트 탑승 중 권총 밀수를 위한 출입국 관리소,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,새로운 유전자 가위 기능 확인 유전 질환 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 삼단 일러스트,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란의 새로운 대공 방어 시스템을 공공 안보를 위한 군사력 강화,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,"북한,유엔 인권결의안에 반대한 모든 대응 조치 강행",6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [25]:
t5['text'] = t5['text'].progress_apply(p2g)
t5['text'] = t5['text'].apply(lambda x: x.replace('.', ''))
t5.head()

  0%|          | 0/4900 [00:00<?, ?it/s]/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 4900/4900 [16:34<00:00,  4.93it/s]


,ID,text,target,url,date
0,ynat-v1_train_04767,해외에서 구입한 요트에서 권총을 밀수하는 출입국 관리소 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,새로운 유전자 가위로 기능 확인 유전 질환 치료를 기대합니다,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐에 있는 신사의 사진,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란의 새로운 대공 방어 체계 개방하고 확보하기 위해 군사력을 증강한다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,그들이 채택한 인권 결의안에 대한 모든 대응책을 강구하는 등 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [26]:
train.to_csv('../data/hanspell_p2g_train_split.csv', index=False)
google.to_csv('../data/hanspell_p2g_backtrans_google.csv', index=False)
t5.to_csv('../data/hanspell_p2g_backtrans_t5.csv', index=False)

In [27]:
merged = pd.concat([train, google])
merged.reset_index(drop=True, inplace=True)
merged

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트 타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공 방어 시스템 공개 안 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한유엔 채택 인권결의에 모든 대응 조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16
...,...,...,...,...,...
9795,ynat-v1_train_03449,특별 자영업자들이 전액 지원금 받기 위해 한 달 만에 수십 건이 접수됐다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.07.02. 오전 10:14
9796,ynat-v1_train_03189,내비게이션 티맵 최건민 무료 개방,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.05. 오전 11:13
9797,ynat-v1_train_04425,여자 농구 대회 칭저우 송로드 프리 마켓 자선 단체,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.26 11:10
9798,ynat-v1_train_06161,조지가 한국 사람이더라도 박근혜가,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.14. 오후 5:17


In [28]:
merged.to_csv('../data/hanspell_p2g_google_train.csv', index=False)

In [29]:
merged2 = pd.concat([train, t5])
merged2.reset_index(drop=True, inplace=True)
merged2

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트 타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공 방어 시스템 공개 안 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한유엔 채택 인권결의에 모든 대응 조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16
...,...,...,...,...,...
9795,ynat-v1_train_03449,특별 자영업자에게 1만 명의 지원금 신청한 달에 1만 원,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.07.02. 오전 10:14
9796,ynat-v1_train_03189,T-navigation Tmap Tran Gung Mi Free 개통,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.05. 오전 11:13
9797,ynat-v1_train_04425,여자 농구의 날 청주 소나무길 벼룩시장 자선 바,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.26 11:10
9798,ynat-v1_train_06161,한국 여성이 살고 있는 조지아에서 박근혜가에게 촛불은,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.14. 오후 5:17


In [30]:
merged2.to_csv('../data/hanspell_p2g_t5_train.csv', index=False)

In [31]:
merged3 = pd.concat([train, google, t5])
merged3.reset_index(drop=True, inplace=True)
merged3

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트 타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공 방어 시스템 공개 안 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한유엔 채택 인권결의에 모든 대응 조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16
...,...,...,...,...,...
14695,ynat-v1_train_03449,특별 자영업자에게 1만 명의 지원금 신청한 달에 1만 원,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.07.02. 오전 10:14
14696,ynat-v1_train_03189,T-navigation Tmap Tran Gung Mi Free 개통,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.05. 오전 11:13
14697,ynat-v1_train_04425,여자 농구의 날 청주 소나무길 벼룩시장 자선 바,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.26 11:10
14698,ynat-v1_train_06161,한국 여성이 살고 있는 조지아에서 박근혜가에게 촛불은,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.14. 오후 5:17


In [32]:
merged3.to_csv('../data/hanspell_p2g_t5_google_train.csv', index=False)